In [ ]:
import torch
import numpy as np
import mediapy
import pickle
import torch
from scipy.io import savemat
import argparse
from tqdm import tqdm
from auxiliar.tracking_viz import generate_frame_image_cv

import matplotlib.pyplot as plt
from src.ortho_factorization import marques_factorization
import rerun as rr
from src.mat_compl import alternating_matrix_completion

%load_ext autoreload
%autoreload 2


In [ ]:
import cv2 as cv
from auxiliar.read_video import read_video_or_images
from src.tapnext_infer import init_tapnext

model_video_size = (256, 256)
device = "cuda:0"

#path = "../mast3r/images_in/casa_bea_4_photos"
path = "../mast3r/images_in/lixo_lab.mp4"
#video_resized, video = read_video_or_images("../mast3r/images_in/bordalo.mp4",model_video_size, device)

video_square, _ = read_video_or_images(path)
print(video_square.shape)
torch.cuda.empty_cache()

In [ ]:
# Design the query points array in the original video size
step = 70
ys, xs = np.meshgrid(np.linspace(8, video_square[0].shape[0]-step, step), np.linspace(8, video_square[0].shape[1]-step, step))
query_points_initial = torch.tensor(
    np.stack(
    [np.zeros(len(xs.flatten())), xs.flatten(), ys.flatten()], axis=1
)[None],dtype=torch.float32).to(device)

#resize the querys points to the model size
query_points_initial[0, :, 1:3] /= torch.tensor([video_square[0].shape[1] / model_video_size[0], video_square[0].shape[0] / model_video_size[1]]).to(device) # 1, N, 3(frames,x,y)

#creata tensors for the video information
frame_cut = max(20, len(video_square))
#frame_cut = 106
frame_skip = 7
video_tensor = torch.cat([(frame).unsqueeze(0) for frame in video_square]).unsqueeze(0)[:,0:frame_cut:frame_skip,...] # 1, time, H, W, 3
video_tensor = torch.clip(video_tensor, -1, 1)
num_frames = video_tensor.shape[1]

print("frames:", num_frames)
torch.cuda.empty_cache()

In [ ]:
from src.tapnext_infer import run_tapnext, init_alltracker
from src.new_queries import add_new_tracks
from auxiliar.read_video import resize_to_max_side
import utils.saveload
import utils.basic
import utils.improc

mode = "cotracker"  # choose: "alltracker", "tapnext", "cotracker"

match mode:
    # ==============================================================
    case "alltracker":

        model = init_alltracker(device)

        rgbs = resize_to_max_side((video_tensor.permute(0,1,4,2,3) + 1) / 2 * 255)
        B, T, C, H, W = rgbs.shape
        
        flows_e, visconf_maps_e, _, _ = model.forward_sliding(
            rgbs[:, 0:], iters=4, sw=None, is_training=False
        )

        # grid in pixel coords
        grid_xy = utils.basic.gridcloud2d(1, H, W, norm=False, device="cpu").float()  # [1,H*W,2]
        grid_xy = grid_xy.permute(0, 2, 1).reshape(1, 1, 2, H, W)  # [1,1,2,H,W]

        # flows_e: [1, T, 2, H, W]
        # grid_xy: [1, 1, 2, H, W]
        traj_maps_e = flows_e + grid_xy  # [1, T, 2, H, W]

        # → [1, T, H, W, 2]
        traj_maps_e = traj_maps_e.permute(0, 1, 3, 4, 2)

        # get forward visibility confidence (channel 1)
        vis_forward = visconf_maps_e[:, :, 1, :, :]  # [1, T, H, W]

        # apply a threshold (e.g. > 0.5)
        visible_mask = vis_forward > 0.75             # [1, T, H, W]
        visible_mask = visible_mask.unsqueeze(-1)    # [1, T, H, W, 1]
        visible_mask = visible_mask.expand_as(traj_maps_e)  # [1, T, H, W, 2]

        traj_maps_e[~visible_mask] = float('nan')


        # get original video shape
        H_orig, W_orig = video_tensor.shape[2:4]
        scale_x = W_orig / W  # width ratio
        scale_y = H_orig / H  # height ratio
        scale = torch.tensor([scale_x, scale_y], device=traj_maps_e.device)

        # apply before flattening
        traj_maps_e = traj_maps_e * scale

        # flatten to [1, T, H*W, 2]
        output_tensor = traj_maps_e.reshape(1, traj_maps_e.shape[1], H * W, 2)  # [1, T, N, 2]

    # ==============================================================
    case "tapnext":
        
        tapnext = init_tapnext(device, "/home/manuelf/storage/bootstapnext_ckpt.npz")
        print("Model initialized")

        track_histories = run_tapnext(
            video_tensor,  # send the video resized to the model size
            query_points_initial,
            tapnext,
            device=device,
            new_tracks_flag=False,  # or None
        )
        output = {}

        for track_id, trajectory in tqdm(track_histories.items()):  # per feature
            coords = []
            for t in range(num_frames):  # per frame
                step = next((pos for (frame, pos) in trajectory if frame == t), None)
                coords.append(step if step is not None else torch.tensor([float('nan'), float('nan')]))
            output[track_id] = torch.stack(coords)  # shape: [num_frames, 2]

        output_list = [trajectory.unsqueeze(0) for _, trajectory in output.items()]
        output_tensor = torch.cat(output_list, dim=0).unsqueeze(0).permute(0, 2, 1, 3)  # [1, num_frames, num_feats, 2]

        # Flip x/y and scale back to original video size
        output_tensor = output_tensor[:, :, :, [1, 0]]
        output_tensor *= torch.tensor([
            video_tensor[0].shape[1] / model_video_size[0],
            video_tensor[0].shape[0] / model_video_size[1]
        ])

        torch.cuda.empty_cache()

    # ==============================================================
    case "cotracker":
        cotracker = torch.hub.load("facebookresearch/co-tracker", "cotracker3_offline").to(device)

        pred_tracks, pred_visibility = cotracker(
            video_tensor.squeeze().permute(0, 3, 1, 2).unsqueeze(0).float().to(device),
            grid_size=step,
            thr=0.9,
        )  # B T N 2,  B T N 1

        pred_tracks = pred_tracks.cpu()
        pred_visibility = pred_visibility.cpu()

        # Ensure shape [1, T, N, 1]
        if pred_visibility.ndim == 3:
            pred_visibility = pred_visibility.unsqueeze(-1)

        # Visibility mask
        visible_mask = pred_visibility.expand_as(pred_tracks)
        pred_tracks_masked = pred_tracks.clone()
        pred_tracks_masked[~visible_mask] = float('nan')

        # Optional flip if you want consistency
        # pred_tracks_masked = pred_tracks_masked[:, :, :, [1, 0]]

        output_tensor = pred_tracks_masked  # [1, num_frames, num_feats, 2]
        torch.cuda.empty_cache()

    # ==============================================================
    case _:
        raise ValueError(f"Unknown tracking mode: {mode}")

# Unified final output: output_tensor → shape [1, num_frames, num_feats, 2]
torch.cuda.empty_cache()


In [ ]:

frame_cut_out = output_tensor.shape[1]

# Calculate the number of visible frames for each track
visible_counts = torch.sum(~torch.isnan(output_tensor[0, :frame_cut_out, :, 0]), dim=0) #shape: #feats

# Filter tracks that are visible for at least 20 frames
visible_tracks_mask = visible_counts >= 1
output_tensor_filtered = output_tensor[:, :frame_cut_out, visible_tracks_mask, :]
output_tensor_filtered.shape

video_tensor_original = video_tensor.to("cpu")


torch.cuda.empty_cache()

In [ ]:
generate_frame_image_cv(video_tensor_original.cpu(), output_tensor[:,:,::10,:])

In [ ]:
import torch
import matplotlib.pyplot as plt
from src.interpolations import get_rgb_at_coords_torch

# output_tensor_filtered: [1, num_frames, num_feats, 2]
num_frames = output_tensor_filtered.shape[1]
num_feats = output_tensor_filtered.shape[2]

# Build observation matrix: [feats*2, frames]
obs_mat_full = torch.full((num_frames * 2, num_feats), float("nan"), device=output_tensor_filtered.device)

for frame in range(num_frames):  # per frame
    obs_mat_full[frame*2, :] = output_tensor_filtered[0, frame, :, 0]  # x
    obs_mat_full[frame*2+1, :] = output_tensor_filtered[0, frame, :, 1]  # y

# remove columns (features) that have any NaN values
valid_columns_mask_no_nan = ~torch.isnan(obs_mat_full).any(dim=0)
obs_mat = obs_mat_full[:, valid_columns_mask_no_nan]

# subsample columns (features) by a factor of 50
#obs_mat = obs_mat[:, ::10]

# remove columns with more than ~50% NaN values
nan_counts = torch.isnan(obs_mat_full).sum(dim=0)
valid_columns_mask_half = nan_counts < (obs_mat_full.shape[0] / 1.3)
obs_mat_half_missed = obs_mat_full[:, valid_columns_mask_half]

# percentage of missing values
missing_percentage = (
    torch.isnan(obs_mat_half_missed).sum().float()
    / (obs_mat_half_missed.numel())
    * 100
).item()

print(f"Percentage of missing values: {missing_percentage:.2f}%")

# plot with torch → numpy just for visualization
#plt.figure(figsize=(12, 6))
#plt.imshow((obs_mat_half_missed), aspect="auto", cmap="gray" )
#plt.show()

In [ ]:
import torch
import matplotlib.pyplot as plt

# obs_mat: [2*frames, feats]
# video: [frames, H, W, C] in [-1, 1] (assumed)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))  # Create a 2x3 grid
axes = axes.flatten()

# Predefined features (convert to torch if you want consistency)
features = torch.tensor([3, 9, 315, 227, 266, 207, 107])
features = torch.tensor([3, 9])
rand_feats = torch.randint(0, obs_mat.shape[1], (20,))

features = torch.cat([features, rand_feats])
print("Selected features:", features.tolist())

# Loop through frames
cnt = 0
for i in torch.linspace(0, video_tensor.cpu().shape[1]-1, 6).to(torch.int):  # num_frames
    # Convert frame for plotting
    frame_img = ((video_tensor[0,i].cpu() + 1) / 2.0)
    axes[cnt].imshow(frame_img)

    axes[cnt].scatter(obs_mat[i*2, features], obs_mat[i*2+1, features], s=6, c="red")
    axes[cnt].set_title(f"Frame {i+1}")
    axes[cnt].axis("off")
    cnt += 1

plt.tight_layout()
plt.show()


In [ ]:
from src.ortho_factorization import costeira_marques
import torch.nn.functional as F


obs_completed, _ = alternating_matrix_completion(obs_mat, mode=0, max_iters=200, rank=3)
# completion + factorization
motion, shape, t = costeira_marques(obs_completed) # rank 3
reconstructed_obs = motion @ shape + t.repeat(1, shape.shape[1])

plt.figure(figsize=(18, 6))

# Plot the completed observation matrix
plt.subplot(1, 3, 1)
plt.imshow(obs_completed / np.nanmax(obs_completed), aspect='auto', cmap='gray')
plt.title("Rank constrained Observation Matrix")
plt.colorbar()

# Plot the original observation matrix
plt.subplot(1, 3, 2)
plt.imshow(obs_mat / np.nanmax(obs_mat), aspect='auto', cmap='gray')
plt.title("Original Observation Matrix")
plt.colorbar()

# Plot the singular values
plt.subplot(1, 3, 3)
singular_values_rank = torch.linalg.svd(obs_completed, full_matrices=False).S
plt.plot(singular_values_rank.cpu().numpy(), "-x", linewidth=2)

singular_values_original = torch.linalg.svd(obs_mat, full_matrices=False).S
plt.plot(singular_values_original.cpu().numpy(), "-o")
plt.legend(["rank_constrained", "Original"])
plt.title("Singular Values")
plt.yscale("log")
plt.xlabel("Index")
plt.ylabel("Value")
plt.grid(True)
plt.tight_layout()


#plot the singular values of the reconstructed
plt.figure(figsize=(10, 6))
singular_values_reconstructed = torch.linalg.svd(reconstructed_obs, full_matrices=False).S
plt.plot(singular_values_reconstructed.cpu().numpy(), "-o", linewidth=2)
plt.yscale("log")
plt.title("Singular Values of Reconstructed Observation Matrix")
plt.xlabel("Index")
plt.ylabel("Value")
plt.grid(True)
plt.show()


In [ ]:
from auxiliar.depth_tensor_viz import k3d_3d_plot
k3d_3d_plot(shape.cpu().numpy())

In [ ]:
import grpc
import os
import sys
import json

sys.path.append('../boxes/vggt/protos')
import pipeline_pb2 as vggt_pb2
import pipeline_pb2_grpc as vggt_pb2_grpc
from aux import wrap_value, unwrap_value
from PIL import Image
import io

image_byte_list = []
# List all files in the directory
if 0:
    
    files = [os.path.join(path, file) for file in os.listdir(path) if os.path.isfile(os.path.join(path, file))]

    for file in sorted(files):
        # Open the file in binary read mode ('rb') and read its entire content
        with open(file, 'rb') as f:
            image_bytes = f.read()
            image_byte_list.append(image_bytes)
            print(f"Read {file}: {len(image_bytes) / (1024 * 1024):.2f} MB")

else:

    # Convert video frames (tensor) to a list of byte arrays
    for frame in video_tensor[0]:  # Assuming video_tensor_original is [1, time, H, W, C]
        # Convert tensor to numpy array and scale to [0, 255]
        frame_np = (frame.numpy() * 255).astype('uint8')
        # Create an Image object
        img = Image.fromarray(frame_np)
        # Save the image to a bytes buffer
        buffer = io.BytesIO()
        img.save(buffer, format="JPEG")
        # Append the bytes to the list
        image_byte_list.append(buffer.getvalue())

config_json = {
        "command": "3d_infer",
        "parameters": {
            "device": "cuda:0", # TODO: implement this
            "conf_vis": 50.0
        }
}

request = vggt_pb2.Envelope(data={"images":wrap_value(image_byte_list)},
                            config_json = json.dumps(config_json))
channel_opt = [('grpc.max_send_message_length', -1), ('grpc.max_receive_message_length', -1)]
channel=grpc.insecure_channel("localhost:8061",options=channel_opt)
estimator_stub = vggt_pb2_grpc.PipelineServiceStub(channel)
response = estimator_stub.Process(request)
channel.close()

#write the glb as a file
glb_file = unwrap_value(response.data["glb_file"])
with open("output.glb", "wb") as f:
    f.write(glb_file)
    

In [ ]:
from auxiliar.depth_tensor_viz import k3d_3d_plot, plot_depth_tensor_grid
wrld_pts = np.load(io.BytesIO(unwrap_value(response.data["world_points"]))).squeeze().reshape(-1,3)
wrld_conf = np.load(io.BytesIO(unwrap_value(response.data["world_points_conf"]))).squeeze().reshape(-1,1)
                    
k3d_3d_plot(wrld_pts[wrld_conf.flatten() > 1, :][::10000].T)

In [ ]:
plot_depth_tensor_grid(torch.tensor(np.load(io.BytesIO(unwrap_value(response.data["depth"])))))

In [ ]:
import io
from src.projective_factorization import build_depth_weighted_matrix, normalize_measurement_matrix

vggt_depth_tensor = torch.tensor(np.load(io.BytesIO(unwrap_value(response.data["depth"]))))  # 1, time, H, W, 1
vggt_conf_tensor = torch.tensor(np.load(io.BytesIO(unwrap_value(response.data["depth_conf"]))))  # 1, time, H, W, 1
instrisics = torch.tensor(np.load(io.BytesIO(unwrap_value(response.data["intrinsic"]))))
extrinsics = torch.tensor(np.load(io.BytesIO(unwrap_value(response.data["extrinsic"]))))
wrld_pts = torch.tensor(np.load(io.BytesIO(unwrap_value(response.data["world_points"])))) # 1, #frame, H, W, 3

# need to resize the observation matrix to the size of the depth tensor
W_vggt = obs_mat.clone()
W_vggt[0::2,:] *=  (vggt_depth_tensor.shape[2] / video_tensor.shape[2])
W_vggt[1::2,:] *=  (vggt_depth_tensor.shape[3] / video_tensor.shape[3])

# average all intrinsic matrices
K_vggt = torch.mean(instrisics, dim=0) # TODO: this needs to be resized due to the tracks
#K = torch.eye(3)

frames_to_use = vggt_depth_tensor.shape[1]
W_depth_vggt, tracks_lambda_vggt = build_depth_weighted_matrix(W_vggt[0:frames_to_use*2,:], vggt_depth_tensor.squeeze()[0:frames_to_use,:],
                                            fx=K_vggt[0,0], fy=K_vggt[1,1], cx=K_vggt[0,2], cy=K_vggt[1,2])

pattern = torch.tensor([1, 1, 0], dtype=torch.bool)
mask = pattern.repeat((W_depth_vggt.shape[0] + len(pattern) - 1) // len(pattern))[:W_depth_vggt.shape[0]]
W_vggt_nohomg = W_depth_vggt[mask, ...]

# Normalize the measurement matrix with isotropic scaling sqrt(3)
Wn, T_list = normalize_measurement_matrix(W_depth_vggt)                     # Wn: [3F, P]

U, S, Vh = torch.linalg.svd(W_depth_vggt, full_matrices=False)
r = 4
Ur = U[:, :r]
Sr = torch.diag(S[:r])
Vr = Vh[:r, :]

Ssqrt = torch.linalg.cholesky(Sr)  
Motion = Ur @ Ssqrt                     
Sshape = Ssqrt @ Vr                # (4,  P) 
W_hat = Motion @ Sshape

#X3d = Sshape[:3, :] / Sshape[3:4, :]
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Plot the normalized measurement matrix
im1 = axes[0].imshow(W_depth_vggt, aspect='auto', cmap='gray', interpolation='none')
axes[0].set_title("Normalized Measurement Matrix")
fig.colorbar(im1, ax=axes[0])

# Plot the singular values in log scale
axes[1].plot(torch.linalg.svd(W_depth_vggt)[1], "-x")
axes[1].set_xlabel("Index")
axes[1].set_title("Log10 Singular Values")
axes[1].set_yscale('log')
axes[1].grid(True)

plt.tight_layout()
plt.show(fig)


# Factorization of the W_hat with the no homogenous coordinates and the lambda

In [ ]:
from src.projective_factorization import metric_upgrade_daq
U, S, V = torch.linalg.svd(W_depth_vggt)

r = 4
Ur = U[:, :r]
Sr = torch.diag(S[:r])
Vr = Vh[:r, :]

Ssqrt = torch.linalg.cholesky(Sr)  
Motion = Ur @ Ssqrt                     
Shape = Ssqrt @ Vr

H, P_e, R_list, t_list = metric_upgrade_daq(Motion, [K_vggt for _ in range(W_depth_vggt.shape[0] // 3)])

X_e = H @ Shape       # Shape = 4×P from your projective factorization
X_e = X_e / X_e[3]
theta = lambda R :torch.arccos(((torch.trace(R) - 1) / 2).clamp(-1, 1))
R_norm = [theta(R) for R in R_list]
R_norm

In [ ]:
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def camera_centers_and_axes(R_list, t_list, axis_len=0.2):
    centers, axes = [], []
    for R, t in zip(R_list, t_list):
        # camera center in world frame
        C = -R.T @ t
        centers.append(C)
        # camera axes in world frame (columns of R)
        axes.append(R.T * axis_len)   # 3×3, each column = world direction of local axis
    return torch.stack(centers), torch.stack(axes)

def plot_cameras_3d(R_list, t_list, X_e=None, axis_len=0.2, text_offset=0.03):
    """
    Plot camera centers and orientation axes in 3D.

    Args:
        R_list: list of (3,3) rotation matrices
        t_list: list of (3,) translations
        X_e: optional (4,P) homogeneous 3D points
        axis_len: length of drawn axes for each camera
        text_offset: offset for axis labels
    """
    assert len(R_list) == len(t_list)
    F = len(R_list)

    # compute camera centers and axis directions in world coords
    centers = []
    axes = []
    for R, t in zip(R_list, t_list):
        C = -R.T @ t               # camera center
        centers.append(C)
        axes.append(R.T * axis_len)  # world directions of local axes
    centers = torch.stack(centers)
    axes = torch.stack(axes)

    # plot
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111, projection='3d')

    # optional points
    if X_e is not None:
        X = X_e[:3] / X_e[3]
        ax.scatter(X[0], X[1], X[2], c='gray', s=2, alpha=0.4, label='points')

    # camera centers
    ax.scatter(centers[:,0], centers[:,1], centers[:,2], c='k', s=25, label='camera centers')

    for i, (C, A) in enumerate(zip(centers, axes)):
        # each column of A is the world direction of local axis
        ax.plot([C[0], C[0]+A[0,0]], [C[1], C[1]+A[0,1]], [C[2], C[2]+A[0,2]], 'r')  # X
        ax.plot([C[0], C[0]+A[1,0]], [C[1], C[1]+A[1,1]], [C[2], C[2]+A[1,2]], 'g')  # Y
        ax.plot([C[0], C[0]+A[2,0]], [C[1], C[1]+A[2,1]], [C[2], C[2]+A[2,2]], 'b')  # Z

        # camera index label near center
        ax.text(C[0], C[1], C[2], f'{i}', color='k', fontsize=8, weight='bold')

    # formatting
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_box_aspect([1,1,1])
    ax.legend()
    plt.tight_layout()
    plt.show()

    return ax

ax = plot_cameras_3d(R_list, t_list,axis_len=0.05)
ax.axis("equal")

#X_cart = X_e[:3] / X_e[3]
#ax.scatter(X_cart[0], X_cart[1], X_cart[2], c='gray', s=2, alpha=0.5, label='3D points')



# Another depth estimator model

In [ ]:
import sys 
sys.path.append("/home/manuelf/UniDepth/")
from unidepth.models import UniDepthV2

device = "cuda:0"
model = UniDepthV2.from_pretrained("lpiccinelli/unidepth-v2-vitl14").to(device) # or "lpiccinelli/unidepth-v1-cnvnxtl" for the ConvNext backbone


In [ ]:
depths = []
uni_conf = []
uni_focal = []

for frame in tqdm(range(video_tensor.shape[1])):
    pred = model.infer(video_tensor[0,frame].permute(2,0,1).to(device))
    #rnd = torch.rand(1).to(device) * 100
    rnd = 1
    #print(rnd)
    depths.append(pred["depth"] * rnd)
    uni_conf.append(1 / pred["confidence"])
    uni_focal.append(pred["intrinsics"])

uni_depth_tensor = torch.stack(depths).unsqueeze(-1).cpu().squeeze()  # (num_frames, H, W, 1)
uni_conf_tensor = torch.stack(uni_conf).unsqueeze(-1).cpu().squeeze()  # (num_frames, H, W, 1)
uni_focal_tensor = torch.stack(uni_focal).cpu().squeeze()  # (num_frames, 3, 3)

In [ ]:
from src.projective_factorization import build_depth_weighted_matrix, normalize_measurement_matrix

# make the tracks fit into the new depth size
W_uni = obs_mat.clone()
W_uni[0::2,:] = W_uni[0::2] * (uni_depth_tensor.shape[1] / video_tensor.shape[2])
W_uni[1::2,:] = W_uni[1::2] * (uni_depth_tensor.shape[2] / video_tensor.shape[3])

K_uni_mean = torch.mean(uni_focal_tensor, dim=0)
#K = torch.eye(3)

frames_to_use = uni_depth_tensor.shape[0]
W_depth_uni, tracks_lambda_unidepth = build_depth_weighted_matrix(W_uni[0:frames_to_use*2,:], uni_depth_tensor[0:frames_to_use,:],
                                            fx=K_uni_mean[0,0], fy=K_uni_mean[1,1], cx=K_uni_mean[0,2], cy=K_uni_mean[1,2])

pattern = torch.tensor([1, 1, 0], dtype=torch.bool)
mask = pattern.repeat((W_depth_uni.shape[0] + len(pattern) - 1) // len(pattern))[:W_depth_uni.shape[0]]
W_uni_nohomg = W_depth_uni[mask, ...]

Wn, T_list = normalize_measurement_matrix(W_depth_uni)                     # Wn: [3F, P]

U, S, Vh = torch.linalg.svd(Wn, full_matrices=False)
r = 4
Ur = U[:, :r]
Sr = torch.diag(S[:r])
Vr = Vh[:r, :]

Ssqrt = torch.linalg.cholesky(Sr)  
Motion = Ur @ Ssqrt                     
Sshape = Ssqrt @ Vr                # (4,  P) 
W_hat = Motion @ Sshape

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Plot the normalized measurement matrix
im1 = axes[0].imshow(W_depth_uni, aspect='auto', cmap='gray', interpolation='none')
axes[0].set_title("Normalized Measurement Matrix")
fig.colorbar(im1, ax=axes[0])

# Plot the singular values in log scale
axes[1].plot(torch.linalg.svd(W_uni_nohomg)[1], "-x")
axes[1].set_xlabel("Index")
#axes[1].set_title("Log10 Singular Values")
axes[1].set_yscale('log')
axes[1].grid(True)

plt.tight_layout()
plt.show(fig)


In [ ]:
from auxiliar.depth_tensor_viz import k3d_3d_plot
k3d_3d_plot(costeira_marques(W_vggt_nohomg)[1]) # tomasi
#plt.imshow(costeira_marques(W_uni_nohomg)[1], aspect='auto', cmap='gray')


In [ ]:
from src.ortho_factorization import calibrate_orthographic, random_affine_camera

if 0:

    F, P = 6, 100
    U = torch.rand(2* F, 4)
    S = torch.diag(torch.tensor([1.0, 1.0, 1.0, 1.0]))  
    V = torch.rand(4, P)
    rand_final = U @ S @ V

    rand_lambda = torch.rand(F, P)
    rand_lambda_hat = rand_lambda.clone()

    # offset in all frames
    rand_lambda_hat[2,:] += 2.0   # deeper in later frames
    rand_lambda_hat[0,:] += 1.0   # deeper in later frames

    rand_lambda_expanded = rand_lambda.repeat_interleave(2, dim=0)
    rand_W = rand_final / rand_lambda_expanded   # observed measurement
    
else:

    F, P = 6, 100
    shape = torch.rand(4, P)

    motion = torch.stack([random_affine_camera() for _ in range(F)]).reshape(-1, 4)  # [2F, 4]
    rand_lambda = random_tensor = (1.2 - 0.5) * torch.rand(F,P) + 0.5

    rand_W = (motion @ shape) / rand_lambda.repeat_interleave(2, dim=0)

    rand_lambda_hat = rand_lambda.clone()

    # add a random offset to each line of rand_lambda_hat
    offsets_list = [torch.rand(1).item() * 5.0 for _ in range(F)]
    for f in range(F):
        rand_lambda_hat[f, :] += offsets_list[f]
    
    


scales, offsets, W, M, first_W, z = calibrate_orthographic(
    rand_W, rand_lambda_hat, K=torch.eye(3),
    iters=10, rank=4, tol=1e-6
)

plt.figure()
plt.plot(torch.linalg.svdvals(first_W)[:7], "-x", label="before")
plt.plot(torch.linalg.svdvals(W)[:7], "-x", label="after")
plt.yscale("log")
plt.legend()

fig, axs = plt.subplots(2, 1, figsize=(10,8), sharex=True)

num_iters, F = scales.shape
iters = torch.arange(num_iters)

# --- subplot 1: scales ---
for f in range(F):
    axs[0].plot(iters, scales[:, f].cpu(), "-o", label=f"frame {f}")

axs[0].set_ylabel("Scale d[f]")
axs[0].set_title("Per-frame scale convergence")
axs[0].grid(True)
axs[0].legend(ncol=3, fontsize=8)

# --- subplot 2: offsets ---
for f in range(F):
    axs[1].plot(iters, offsets[:, f].cpu()/scales[:, f].cpu(), "-o", label=f"frame {f}")
    axs[1].hlines(-offsets_list[f], 0, iters[-1])

axs[1].set_xlabel("Iteration")
axs[1].set_ylabel("Offset s[f]")
axs[1].set_title("Per-frame offset convergence")
axs[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
import torch

def fit_scales_offsets_spectral_tail(W, lam_hat, r=4, iters=1000,
                                     lr=1e-2, alpha=1e-2, beta=1e-3, device=None):
    """
    W:       [2F,P]
    lam_hat: [F,P]
    Returns s,t and the corrected Y(s,t).
    """
    if device is None:
        device = W.device
    F, P = lam_hat.shape
    assert W.shape[0] == 2*F and W.shape[1] == P

    # parameters with safe reparameterization
    s_raw = torch.zeros(F, device=device, requires_grad=True)  # s = 1 + softplus(s_raw)
    t_raw = torch.zeros(F, device=device, requires_grad=True)  # t =     softplus/tanh choice

    opt = torch.optim.Adam([s_raw, t_raw], lr=lr)

    def build_Y():
        s = 1.0 + torch.nn.functional.softplus(s_raw)          # keep s >= 1
        t = 0.1 * torch.tanh(t_raw)                            # keep t in ~[-0.1,0.1] (adjust)
        lam_corr = (s[:,None]*lam_hat + t[:,None]).clamp_min(1e-6)
        Y = lam_corr.repeat_interleave(2, dim=0) * W
        return Y, s, t

    best = None
    for _ in range(iters):
        opt.zero_grad()
        Y, s, t = build_Y()

        # singular values
        S = torch.linalg.svdvals(Y)
        tail = S[r:]                                           # σ_{r+1..}
        loss_tail = (tail**2).sum()

        # gentle priors to prevent "smashing scales"
        loss_reg = alpha * ((s-1)**2).sum() + beta * (t**2).sum()

        loss = loss_tail + loss_reg
        loss.backward()
        opt.step()

        # keep best by tail energy
        with torch.no_grad():
            if best is None or loss_tail.item() < best[0]:
                best = (loss_tail.item(), s.detach().clone(), t.detach().clone(), Y.detach().clone())

    _, s_best, t_best, Y_best = best
    return s_best, t_best, Y_best


F, P = 6, 100
U = torch.randn(2*F, 4, dtype=torch.float32)
S = torch.diag(torch.tensor([1.0, 1.0, 1.0, 1.0], dtype=torch.float32))  
V = torch.randn(4, P, dtype=torch.float32)
rand_final = U @ S @ V

rand_lambda = torch.rand(F, P)
rand_lambda_hat = rand_lambda.clone()

# scale perturbation
#rand_lambda_hat[-1,:] *= 3.0  # frame-10 deeper (true λ=2)

# offset perturbation
rand_lambda_hat[3,:] += 3.0   # frame-4 closer
rand_lambda_hat[1,:] -= 10.0  # frame-4 closeewr

rand_lambda_expanded = rand_lambda.repeat_interleave(2, dim=0)
rand_W = rand_final / rand_lambda_expanded   # observed measurement

s_est, t_est, Y_corr = fit_scales_offsets_spectral_tail(rand_W, rand_lambda_hat, r=4, iters=1000, lr=1)

sv_before = torch.linalg.svdvals((rand_lambda_hat.repeat_interleave(2,0))*rand_W)
sv_after  = torch.linalg.svdvals(Y_corr)
print("σ5 before:", sv_before[4].item())
print("σ5 after :", sv_after[4].item())
print("s:", s_est)
print("t:", t_est)

plt.figure(figsize=(10, 6))
plt.plot(sv_before.cpu().numpy(), "-x", label="before")
plt.plot(sv_after.cpu().numpy(), "-o", label="after")
plt.yscale("log")
plt.legend()
plt.title("Singular Values")
plt.xlabel("Index")
plt.ylabel("Value")
plt.grid(True)
plt.show()


In [ ]:
from auxiliar.depth_tensor_viz import k3d_3d_plot

k3d_3d_plot(costeira_marques(res.A)[1])
#k3d_3d_plot(costeira_marques(W_uni_nohomg)[1])

# Tests

In [ ]:
s_vggt = torch.linalg.svdvals(W_vggt_nohomg)
plt.plot(s_vggt[:20] /  s_vggt[0], "-o", label="vggt")

s_unidepth = torch.linalg.svdvals(W_uni_nohomg)
plt.plot(s_unidepth[:20] / s_unidepth[0], "-x", label="unidepth")

plt.yscale("log")
plt.legend()
plt.grid()
plt.title("Singular Values Comparison VGGt vs UniDepth, normalized")

In [ ]:
plt.plot(tracks_lambda_vggt[0,:].flatten() / torch.max(tracks_lambda_vggt[0,:].flatten()), label="vggt")
plt.plot(tracks_lambda_unidepth[0,:].flatten() / torch.max(tracks_lambda_unidepth[0,:].flatten()), label="unidepth")
plt.legend()

In [ ]:
%matplotlib inline

scales = []
offsets = []


for frame in range(5, tracks_lambda_vggt.shape[0]):
    x = tracks_lambda_vggt[frame, :].flatten()
    y = tracks_lambda_unidepth[frame, :].flatten()
    A = torch.stack([x, torch.ones_like(x)], dim=1)  # [N,2]
    sol = torch.linalg.lstsq(A, y.unsqueeze(1)).solution
    s, o = sol.squeeze()

    scales.append(s.item())
    offsets.append(o.item())

    y_pred = s * x + o
    residuals = y - y_pred
    rmse = torch.sqrt(torch.mean(residuals**2))
    corr = torch.corrcoef(torch.stack([x, y]))[0, 1]
    print(f"RMSE: {rmse.item():.4f}, correlation: {corr.item():.4f}")


    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    # Left subplot: Video frame with residuals overlay
    axes[0].imshow((video_tensor_original[0,frame] + 1) / 2)
    xs = obs_mat[frame * 2 + 0, :]
    ys = obs_mat[frame * 2 + 1, :]
    scatter = axes[0].scatter(xs.flatten(), ys.flatten(), c=np.abs(residuals.flatten()), cmap='hot', s=10)
    fig.colorbar(scatter, ax=axes[0], label='Residuals')
    axes[0].set_title("Residuals Visualization")

    # Right subplot: Residuals plot
    axes[1].plot(residuals.numpy(), label="Residuals")
    axes[1].set_title("Residuals Plot")
    axes[1].set_xlabel("Feature Index")
    axes[1].set_ylabel("Residual Value")
    axes[1].legend()

    plt.tight_layout()
    plt.show()



In [ ]:
uni_depth_tensor.shape

In [ ]:
from src.projective_factorization import sample_depths

def backproject_points(tracks, depths, fx, fy, cx, cy):
    """
    tracks: [2F, P] pixel coords
    depths: [F, P] sampled depths for each feature
    Returns: [F, P, 3] 3D points in camera coords per frame
    """
    F, P = depths.shape
    u = tracks[0::2, :]  # [F,P]
    v = tracks[1::2, :]  # [F,P]

    X = (u - cx) / fx * depths
    Y = (v - cy) / fy * depths
    Z = depths

    return torch.stack([X, Y, Z], dim=-1)  # [F,P,3]

num_frames = obs_mat_depth_sized.shape[0] // 2

tracks_depth = sample_depths(depth_tensor[0:num_frames,...], obs_mat_depth_sized[0:num_frames*2,...])  # [F, P]

K = torch.mean(instrisics, dim=0)

pts = backproject_points(obs_mat_depth_sized[0:num_frames*2,...], tracks_depth,
                          fx=K[0,0], fy=K[1,1], cx=K[0,2], cy=K[1,2]) # all frames

X,Y,Z = pts[:,0].cpu(), pts[:,1].cpu(), pts[:,2].cpu()

#use rerrun to see the pts
rr.init("3d_backprojection")
rr.notebook_show(width=int(1920*0.75), height=int(1080*0.75))
rr.log("world/3d_completed", rr.Points3D(pts))

In [ ]:
from auxiliar.depth_tensor_viz import plot_depth_tensor_grid
plot_depth_tensor_grid(vggt_depth_tensor)

In [ ]:
# Find fundamental matrices between frames
from tqdm import tqdm

fig, axes = plt.subplots(3,2, figsize=(15, 10))  # Create a 2x3 grid
axes = axes.flatten()

for frame in range(obs_mat.shape[0] // 2 - 1):

    pts1 = obs_mat[frame*2:frame*2+2, :].T
    pts2 = obs_mat[(frame+8)*2:(frame+8)*2+2, :].T

    F, mask = cv.findFundamentalMat(
        pts1.cpu().numpy(), 
        pts2.cpu().numpy(), 
        cv.FM_RANSAC,
        ransacReprojThreshold=1.0)
    
    # show both imaegs side by side and represent the outliers
    img1 = ((video[frame] + 1) / 2.0)
    img2 = ((video[frame+8] + 1) / 2.0)
    combined_img = np.hstack((img1, img2))
    axes[frame].imshow(combined_img)

    for i in range(0, pts1.shape[0], 10):
        if mask[i]:
            color = 'g'  # Inlier
        else:
            color = 'r'  # Outlier
        axes[frame].scatter(pts1[i, 0], pts1[i, 1], s=6, c=color)
        axes[frame].scatter(pts2[i, 0] + img1.shape[1], pts2[i, 1], s=6, c=color)  # Offset x for second image
        axes[frame].set_title(f"Frame {frame+1} and {frame+2}")

    break  # only first pair for demo
